In [80]:
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
#try within same table encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [81]:
titanictrain = pd.read_csv('train.csv')

In [82]:
titanictrain.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [83]:
titanictest = pd.read_csv('test.csv')

In [84]:
titanictest.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [85]:
# I will check number of missing data in each attribute in trainset with 891 passenger
titanictrain.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [86]:
titanictest.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [87]:
#we can see from results that 3 columns have missing data: age, cabin: a lot missing and embarked
#i will try to substitute the missing values with the mean of the data for age and mode? for the embarked,
#cabin number can maybe be replaced starting with letter B is some sector of the ship that can impact the surviving chances, and
#in the same cell if there are more cabin names the all start with same letter, we can maybe choose just the first letter

#i will drop column ticket number because i don't find it as having an infulence to chances to survive
#inside the name column there are signs like 'the second' as nobel, but in train set one did survive and one didn't so i will
#treat it as a significant impact on surviving chances
#also, I will later remove PassingerID with no influence on surviving 


In [88]:
#i will check on datatypes of columns
titanictrain.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [89]:
#titanictrain = titanictrain.drop(columns =['Name', 'Ticket'])

titanictrain = titanictrain.drop(['Name', 'Ticket'],axis=1)

#nisam stavila za minjat originalnu tabelu , axis=! javlja dole gresku ali radi! dole vidis da izbaci kolone
#inplace=True
# Remove two columns name is 'C' and 'D'
#df.drop(['C', 'D'], axis=1)
#https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/


In [90]:
titanictrain.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S
5,6,0,3,male,NaN,0,0,8.4583,NaN,Q
6,7,0,1,male,54.0,0,0,51.8625,E46,S
7,8,0,3,male,2.0,3,1,21.0750,NaN,S
8,9,1,3,female,27.0,0,2,11.1333,NaN,S
9,10,1,2,female,14.0,1,0,30.0708,NaN,C


In [91]:
titanictrain['CabinSector'] = titanictrain['Cabin'].str.slice(0,1)

titanictrain.head(25)
#i took only the first letter of the cabin to create and add column cabin sector
#this is just to show how i would do that, but if i replace naan with any mode?? or median i am
#afraid to affect on this feature too much, because more around 


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,male,22.0,1,0,7.2500,NaN,S,NaN
1,2,1,1,female,38.0,1,0,71.2833,C85,C,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S,NaN
3,4,1,1,female,35.0,1,0,53.1000,C123,S,C
4,5,0,3,male,35.0,0,0,8.0500,NaN,S,NaN
5,6,0,3,male,NaN,0,0,8.4583,NaN,Q,NaN
6,7,0,1,male,54.0,0,0,51.8625,E46,S,E
7,8,0,3,male,2.0,3,1,21.0750,NaN,S,NaN
8,9,1,3,female,27.0,0,2,11.1333,NaN,S,NaN
9,10,1,2,female,14.0,1,0,30.0708,NaN,C,NaN


In [92]:
# i will replace cabinsector column with random values of sectors i notices in train dataset:ABCDEFG, i CAN
#USE cABINsECTOR COLUMN INSTEAD OF CABIN OR REPLACE VALUES OF CABIN WITH ACBINSECTOR
np.random.seed(123)
titanictrain.loc[titanictrain['CabinSector'].isna(), 'CabinSector'] = [i for i in np.random.choice(['A', 'B', 'C', 'D','E', 'F','G'], titanictrain['CabinSector'].isna().sum())]


In [93]:
titanictrain.head(25)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,male,22.0,1,0,7.2500,NaN,S,G
1,2,1,1,female,38.0,1,0,71.2833,C85,C,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S,F
3,4,1,1,female,35.0,1,0,53.1000,C123,S,C
4,5,0,3,male,35.0,0,0,8.0500,NaN,S,G
5,6,0,3,male,NaN,0,0,8.4583,NaN,Q,C
6,7,0,1,male,54.0,0,0,51.8625,E46,S,E
7,8,0,3,male,2.0,3,1,21.0750,NaN,S,E
8,9,1,3,female,27.0,0,2,11.1333,NaN,S,C
9,10,1,2,female,14.0,1,0,30.0708,NaN,C,G


In [94]:
len(titanictrain['Cabin'])

891

In [95]:
len(titanictrain['CabinSector'])

891

In [96]:
#copies column CabinSector into new column Cabin

titanictrain.loc[:,'Cabin'] = (titanictrain['CabinSector'])

In [97]:
titanictrain.head(25)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,male,22.0,1,0,7.2500,G,S,G
1,2,1,1,female,38.0,1,0,71.2833,C,C,C
2,3,1,3,female,26.0,0,0,7.9250,F,S,F
3,4,1,1,female,35.0,1,0,53.1000,C,S,C
4,5,0,3,male,35.0,0,0,8.0500,G,S,G
5,6,0,3,male,NaN,0,0,8.4583,C,Q,C
6,7,0,1,male,54.0,0,0,51.8625,E,S,E
7,8,0,3,male,2.0,3,1,21.0750,E,S,E
8,9,1,3,female,27.0,0,2,11.1333,C,S,C
9,10,1,2,female,14.0,1,0,30.0708,G,C,G


In [98]:
#**********
#cabinsector nan solved with filling with random values and i will fill age and fare(isnull on test checked before)with median
#and embarked with mode because of letter

In [99]:
medianagetrain = titanictrain['Age'].astype('float64').median(axis=0)
titanictrain['Age'].replace(np.nan, medianagetrain, inplace=True)

In [100]:
medianfaretrain = titanictrain['Fare'].astype('float64').median(axis=0)
titanictrain['Fare'].replace(np.nan, medianfaretrain, inplace=True)

In [101]:
modeembarkedtrain = titanictrain['Embarked'].astype('string').mode
titanictrain['Embarked'].replace(np.nan, modeembarkedtrain, inplace=True)

In [102]:
titanictrain.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
CabinSector    0
dtype: int64

In [103]:
titanictrain.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,male,22.0,1,0,7.2500,G,S,G
1,2,1,1,female,38.0,1,0,71.2833,C,C,C
2,3,1,3,female,26.0,0,0,7.9250,F,S,F
3,4,1,1,female,35.0,1,0,53.1000,C,S,C
4,5,0,3,male,35.0,0,0,8.0500,G,S,G
5,6,0,3,male,28.0,0,0,8.4583,C,Q,C
6,7,0,1,male,54.0,0,0,51.8625,E,S,E
7,8,0,3,male,2.0,3,1,21.0750,E,S,E
8,9,1,3,female,27.0,0,2,11.1333,C,S,C
9,10,1,2,female,14.0,1,0,30.0708,G,C,G


In [104]:
titanictrainohe = pd.get_dummies(data=titanictrain, columns=['Sex','Cabin','Embarked'])
#drop_first da 1 kolona,tj. 0 i 1, male i ako nije onda female ka ako odbacis prvu opciju, onda druga,inace ti 
#nastanu 2 kolone
#crated new table with encoded columns instead of exsisting, but i could now drop the column s
#passinger id and CabinSector

In [105]:
titanictrainohe.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,CabinSector,Sex_female,Sex_male,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,"Embarked_<bound method Series.mode of 0 S\n1 C\n2 S\n3 S\n4 S\n ..\n886 S\n887 S\n888 S\n889 C\n890 Q\nName: Embarked, Length: 891, dtype: string>",Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,G,0,1,...,0,0,0,0,1,0,0,0,0,1
1,2,1,1,38.0,1,0,71.2833,C,1,0,...,1,0,0,0,0,0,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,F,1,0,...,0,0,0,1,0,0,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,C,1,0,...,1,0,0,0,0,0,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,G,0,1,...,0,0,0,0,1,0,0,0,0,1


In [106]:
titanictrain.head(5)
#titanictrain dataset stays the same after onehotencoding

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,male,22.0,1,0,7.2500,G,S,G
1,2,1,1,female,38.0,1,0,71.2833,C,C,C
2,3,1,3,female,26.0,0,0,7.9250,F,S,F
3,4,1,1,female,35.0,1,0,53.1000,C,S,C
4,5,0,3,male,35.0,0,0,8.0500,G,S,G


In [107]:
#*************
#there was after encoding en error saying embarked has to be string or numeric,median in nan made cells 
#different types so i turned whole column in string
titanictrain = titanictrain.astype({'Embarked': str})

In [108]:
#another way
encodecols= ['Sex', 'Cabin', 'Embarked']

for col in encodecols:
    titanictrain[col]=le.fit_transform(titanictrain[col])
    print(le.classes_)
    
titanictrain.head(5)
                                

['female' 'male']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T']
['<bound method Series.mode of 0      S\n1      C\n2      S\n3      S\n4      S\n      ..\n886    S\n887    S\n888    S\n889    C\n890    Q\nName: Embarked, Length: 891, dtype: string>'
 'C' 'Q' 'S']


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,1,0,3,1,22.0,1,0,7.2500,6,3,G
1,2,1,1,0,38.0,1,0,71.2833,2,1,C
2,3,1,3,0,26.0,0,0,7.9250,5,3,F
3,4,1,1,0,35.0,1,0,53.1000,2,3,C
4,5,0,3,1,35.0,0,0,8.0500,6,3,G


In [109]:
titanictrain = titanictrain.drop(['CabinSector'],axis=1)

In [110]:
titanictrain.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,7.2500,6,3
1,2,1,1,0,38.0,1,0,71.2833,2,1
2,3,1,3,0,26.0,0,0,7.9250,5,3
3,4,1,1,0,35.0,1,0,53.1000,2,3
4,5,0,3,1,35.0,0,0,8.0500,6,3
5,6,0,3,1,28.0,0,0,8.4583,2,2
6,7,0,1,1,54.0,0,0,51.8625,4,3
7,8,0,3,1,2.0,3,1,21.0750,4,3
8,9,1,3,0,27.0,0,2,11.1333,2,3
9,10,1,2,0,14.0,1,0,30.0708,6,1


In [111]:
titanictrain = titanictrain.drop(['PassengerId'],axis=1)

In [112]:
titanictrain.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,6,3
1,1,1,0,38.0,1,0,71.2833,2,1
2,1,3,0,26.0,0,0,7.9250,5,3
3,1,1,0,35.0,1,0,53.1000,2,3
4,0,3,1,35.0,0,0,8.0500,6,3
5,0,3,1,28.0,0,0,8.4583,2,2
6,0,1,1,54.0,0,0,51.8625,4,3
7,0,3,1,2.0,3,1,21.0750,4,3
8,1,3,0,27.0,0,2,11.1333,2,3
9,1,2,0,14.0,1,0,30.0708,6,1


In [113]:
#  MinMaxScaler za age and fare
mms = MinMaxScaler()

In [114]:
# fit scaler on training data
titanictrain[['Age','Fare']] = mms.fit_transform(titanictrain[['Age','Fare']])

In [115]:
# fit & transform training data za age i fare 


In [116]:
titanictrain.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,0.271174,1,0,0.014151,6,3
1,1,1,0,0.472229,1,0,0.139136,2,1
2,1,3,0,0.321438,0,0,0.015469,5,3
3,1,1,0,0.434531,1,0,0.103644,2,3
4,0,3,1,0.434531,0,0,0.015713,6,3
5,0,3,1,0.346569,0,0,0.016510,2,2
6,0,1,1,0.673285,0,0,0.101229,4,3
7,0,3,1,0.019854,3,1,0.041136,4,3
8,1,3,0,0.334004,0,2,0.021731,2,3
9,1,2,0,0.170646,1,0,0.058694,6,1


In [117]:
#titanictest[['Age','Fare']] = mms.transform(titanictest[['Age','Fare']])
#****
#probala i tocno, za titanictest samo transform koristis, jer se mms objekt primijeni koji fit po traindata

In [118]:
#******TEST**********

In [119]:
#i will check on datatypes of columns
titanictest.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [120]:
#titanictrain = titanictrain.drop(columns =['Name', 'Ticket'])
titanictest = titanictest.drop(['Name', 'Ticket'],axis=1)

#nisam stavila za minjat originalnu tabelu , axis=! javlja dole gresku ali radi! dole vidis da izbaci kolone
#inplace=True
# Remove two columns name is 'C' and 'D'
#df.drop(['C', 'D'], axis=1)
#https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/

In [121]:
titanictest.head(15)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,male,34.5,0,0,7.8292,NaN,Q
1,893,3,female,47.0,1,0,7.0000,NaN,S
2,894,2,male,62.0,0,0,9.6875,NaN,Q
3,895,3,male,27.0,0,0,8.6625,NaN,S
4,896,3,female,22.0,1,1,12.2875,NaN,S
5,897,3,male,14.0,0,0,9.2250,NaN,S
6,898,3,female,30.0,0,0,7.6292,NaN,Q
7,899,2,male,26.0,1,1,29.0000,NaN,S
8,900,3,female,18.0,0,0,7.2292,NaN,C
9,901,3,male,21.0,2,0,24.1500,NaN,S


In [122]:
titanictest['CabinSector'] = titanictest['Cabin'].str.slice(0,1)

titanictest.head(15)
#i took only the first letter of the cabin to create and add column cabin sector
#this is just to show how i would do that, but if i replace naan with any mode?? or median i am
#afraid to affect on this feature too much, because more around 


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,male,34.5,0,0,7.8292,NaN,Q,NaN
1,893,3,female,47.0,1,0,7.0000,NaN,S,NaN
2,894,2,male,62.0,0,0,9.6875,NaN,Q,NaN
3,895,3,male,27.0,0,0,8.6625,NaN,S,NaN
4,896,3,female,22.0,1,1,12.2875,NaN,S,NaN
5,897,3,male,14.0,0,0,9.2250,NaN,S,NaN
6,898,3,female,30.0,0,0,7.6292,NaN,Q,NaN
7,899,2,male,26.0,1,1,29.0000,NaN,S,NaN
8,900,3,female,18.0,0,0,7.2292,NaN,C,NaN
9,901,3,male,21.0,2,0,24.1500,NaN,S,NaN


In [123]:
# i will replace cabinsector column with random values of sectors i notices in train dataset:ABCDEFG, i CAN
#USE cABINsECTOR COLUMN INSTEAD OF CABIN OR REPLACE VALUES OF CABIN WITH ACBINSECTOR
np.random.seed(123)
titanictest.loc[titanictest['CabinSector'].isna(), 'CabinSector'] = [i for i in np.random.choice(['A', 'B', 'C', 'D','E', 'F','G'], titanictest['CabinSector'].isna().sum())]


In [124]:
titanictest.head(25)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,male,34.5,0,0,7.8292,NaN,Q,G
1,893,3,female,47.0,1,0,7.0000,NaN,S,F
2,894,2,male,62.0,0,0,9.6875,NaN,Q,G
3,895,3,male,27.0,0,0,8.6625,NaN,S,C
4,896,3,female,22.0,1,1,12.2875,NaN,S,E
5,897,3,male,14.0,0,0,9.2250,NaN,S,C
6,898,3,female,30.0,0,0,7.6292,NaN,Q,G
7,899,2,male,26.0,1,1,29.0000,NaN,S,B
8,900,3,female,18.0,0,0,7.2292,NaN,C,D
9,901,3,male,21.0,2,0,24.1500,NaN,S,C


In [125]:
len(titanictest['Cabin'])

418

In [126]:
len(titanictest['CabinSector'])

418

In [127]:
#copies column CabinSector into new column Cabin

titanictest.loc[:,'Cabin'] = (titanictest['CabinSector'])

In [128]:
titanictest.head(15)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,male,34.5,0,0,7.8292,G,Q,G
1,893,3,female,47.0,1,0,7.0000,F,S,F
2,894,2,male,62.0,0,0,9.6875,G,Q,G
3,895,3,male,27.0,0,0,8.6625,C,S,C
4,896,3,female,22.0,1,1,12.2875,E,S,E
5,897,3,male,14.0,0,0,9.2250,C,S,C
6,898,3,female,30.0,0,0,7.6292,G,Q,G
7,899,2,male,26.0,1,1,29.0000,B,S,B
8,900,3,female,18.0,0,0,7.2292,D,C,D
9,901,3,male,21.0,2,0,24.1500,C,S,C


In [129]:
print(titanictest)

     PassengerId  Pclass     Sex   Age  SibSp  Parch      Fare Cabin Embarked  \
0            892       3    male  34.5      0      0    7.8292     G        Q   
1            893       3  female  47.0      1      0    7.0000     F        S   
2            894       2    male  62.0      0      0    9.6875     G        Q   
3            895       3    male  27.0      0      0    8.6625     C        S   
4            896       3  female  22.0      1      1   12.2875     E        S   
..           ...     ...     ...   ...    ...    ...       ...   ...      ...   
413         1305       3    male   NaN      0      0    8.0500     G        S   
414         1306       1  female  39.0      0      0  108.9000     C        C   
415         1307       3    male  38.5      0      0    7.2500     A        S   
416         1308       3    male   NaN      0      0    8.0500     G        S   
417         1309       3    male   NaN      1      1   22.3583     B        C   

    CabinSector  
0        

In [130]:
 #**********
#cabinsector nan solved with filling with random values and i will fill age and fare(isnull on test checked before)with median
#and embarked with mode because of letter

In [131]:
medianagetest = titanictest['Age'].astype('float64').median(axis=0)
titanictest['Age'].replace(np.nan, medianagetest, inplace=True)

In [132]:
medianfaretest = titanictest['Fare'].astype('float64').median(axis=0)
titanictest['Fare'].replace(np.nan, medianfaretest, inplace=True)

In [133]:
modeembarkedtest = titanictest['Embarked'].astype('string').mode
titanictest['Embarked'].replace(np.nan, modeembarkedtest, inplace=True)

In [134]:
titanictest.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
CabinSector    0
dtype: int64

In [135]:
titanictest.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,male,34.5,0,0,7.8292,G,Q,G
1,893,3,female,47.0,1,0,7.0000,F,S,F
2,894,2,male,62.0,0,0,9.6875,G,Q,G
3,895,3,male,27.0,0,0,8.6625,C,S,C
4,896,3,female,22.0,1,1,12.2875,E,S,E
5,897,3,male,14.0,0,0,9.2250,C,S,C
6,898,3,female,30.0,0,0,7.6292,G,Q,G
7,899,2,male,26.0,1,1,29.0000,B,S,B
8,900,3,female,18.0,0,0,7.2292,D,C,D
9,901,3,male,21.0,2,0,24.1500,C,S,C


In [136]:
titanictestohe = pd.get_dummies(data=titanictest, columns=['Sex','Cabin','Embarked'])
#drop_first da 1 kolona,tj. 0 i 1, male i ako nije onda female ka ako odbacis prvu opciju, onda druga,inace ti 
#nastanu 2 kolone
#one hot encoding
#crated new table with encoded columns instead of exsisting, but i could now drop the column s
#passinger id and CabinSector

In [137]:
titanictestohe.head(5)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,CabinSector,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,G,0,1,0,0,0,0,0,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,F,1,0,0,0,0,0,0,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,G,0,1,0,0,0,0,0,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,C,0,1,0,0,1,0,0,0,0,0,0,1
4,896,3,22.0,1,1,12.2875,E,1,0,0,0,0,0,1,0,0,0,0,1


In [138]:
titanictest.head(5)
#titanictrain dataset stays the same after onehotencoding

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,male,34.5,0,0,7.8292,G,Q,G
1,893,3,female,47.0,1,0,7.0000,F,S,F
2,894,2,male,62.0,0,0,9.6875,G,Q,G
3,895,3,male,27.0,0,0,8.6625,C,S,C
4,896,3,female,22.0,1,1,12.2875,E,S,E


In [139]:
#*************
#there was after encoding en error saying embarked has to be string or numeric,median in nan made cells 
#different types so i turned whole column in string
titanictest = titanictest.astype({'Embarked': str})

In [140]:
#another way
encodecols= ['Sex', 'Cabin', 'Embarked']

for col in encodecols:
    titanictest[col]=le.fit_transform(titanictest[col])
    print(le.classes_)
    
titanictest.head(5)

['female' 'male']
['A' 'B' 'C' 'D' 'E' 'F' 'G']
['C' 'Q' 'S']


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,CabinSector
0,892,3,1,34.5,0,0,7.8292,6,1,G
1,893,3,0,47.0,1,0,7.0000,5,2,F
2,894,2,1,62.0,0,0,9.6875,6,1,G
3,895,3,1,27.0,0,0,8.6625,2,2,C
4,896,3,0,22.0,1,1,12.2875,4,2,E


In [141]:
titanictest = titanictest.drop(['CabinSector'],axis=1)

In [142]:
titanictest.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,1,34.5,0,0,7.8292,6,1
1,893,3,0,47.0,1,0,7.0000,5,2
2,894,2,1,62.0,0,0,9.6875,6,1
3,895,3,1,27.0,0,0,8.6625,2,2
4,896,3,0,22.0,1,1,12.2875,4,2
5,897,3,1,14.0,0,0,9.2250,2,2
6,898,3,0,30.0,0,0,7.6292,6,1
7,899,2,1,26.0,1,1,29.0000,1,2
8,900,3,0,18.0,0,0,7.2292,3,0
9,901,3,1,21.0,2,0,24.1500,2,2


In [143]:
titanictest = titanictest.drop(['PassengerId'],axis=1)

In [144]:
titanictest.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,34.5,0,0,7.8292,6,1
1,3,0,47.0,1,0,7.0000,5,2
2,2,1,62.0,0,0,9.6875,6,1
3,3,1,27.0,0,0,8.6625,2,2
4,3,0,22.0,1,1,12.2875,4,2
5,3,1,14.0,0,0,9.2250,2,2
6,3,0,30.0,0,0,7.6292,6,1
7,2,1,26.0,1,1,29.0000,1,2
8,3,0,18.0,0,0,7.2292,3,0
9,3,1,21.0,2,0,24.1500,2,2


In [145]:
#  MinMaxScaler za age and fare
#mms = MinMaxScaler()

In [146]:
# fit scaler on training data
#titanictrain[['Age','Fare']] = mms.fit_transform(titanictrain[['Age','Fare']])
titanictest[['Age','Fare']] = mms.transform(titanictest[['Age','Fare']])

In [147]:
# fit & transform training data za age i fare ali fit po train opet pa onda primjena
#nije radilo do import train_test split


In [148]:
titanictest.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,0.428248,0,0,0.015282,6,1
1,3,0,0.585323,1,0,0.013663,5,2
2,2,1,0.773813,0,0,0.018909,6,1
3,3,1,0.334004,0,0,0.016908,2,2
4,3,0,0.271174,1,1,0.023984,4,2
5,3,1,0.170646,0,0,0.018006,2,2
6,3,0,0.371701,0,0,0.014891,6,1
7,2,1,0.321438,1,1,0.056604,1,2
8,3,0,0.220910,0,0,0.014110,3,0
9,3,1,0.258608,2,0,0.047138,2,2


In [149]:
#titanictest[['Age','Fare']] = mms.transform(titanictest[['Age','Fare']])
#****
#probala i tocno, za titanictest samo transform koristis, jer se mms objekt primijeni koji fit po traindata

In [150]:
titanictrain.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,0.271174,1,0,0.014151,6,3
1,1,1,0,0.472229,1,0,0.139136,2,1
2,1,3,0,0.321438,0,0,0.015469,5,3
3,1,1,0,0.434531,1,0,0.103644,2,3
4,0,3,1,0.434531,0,0,0.015713,6,3
5,0,3,1,0.346569,0,0,0.016510,2,2
6,0,1,1,0.673285,0,0,0.101229,4,3
7,0,3,1,0.019854,3,1,0.041136,4,3
8,1,3,0,0.334004,0,2,0.021731,2,3
9,1,2,0,0.170646,1,0,0.058694,6,1


In [151]:
#now I will split train data in train and validation data, because I dont have Y in test data,survived

In [152]:
#iz analytics vidhya:
#https://www.analyticsvidhya.com/blog/2021/04/sklearn-objects-fit-vs-transform-vs-fit_transform-vs-predict-in-scikit-learn/

In [153]:
#prvo Y za izvuci survived kolonu,jer posli kad je izbacis, izbaci se iz svih gornji verzija titanictraina
#nema mi je odakle izvuc

In [154]:
#Y=titanictrain['Survived']; svi redovi i samo 1. kolona
Y=titanictrain.iloc[:,0]
#Y = titanictrain['Survived']
print(Y)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [155]:
#; svi redovi i sve osim 1. kolone

#X= titanictrain.drop(['Survived'],axis=1)
X=titanictrain.iloc[:,1:]
print(X)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked
0         3    1  0.271174      1      0  0.014151      6         3
1         1    0  0.472229      1      0  0.139136      2         1
2         3    0  0.321438      0      0  0.015469      5         3
3         1    0  0.434531      1      0  0.103644      2         3
4         3    1  0.434531      0      0  0.015713      6         3
..      ...  ...       ...    ...    ...       ...    ...       ...
886       2    1  0.334004      0      0  0.025374      0         3
887       1    0  0.233476      0      0  0.058556      1         3
888       3    0  0.346569      1      2  0.045771      0         3
889       1    1  0.321438      0      0  0.058556      2         1
890       3    1  0.396833      0      0  0.015127      5         2

[891 rows x 8 columns]


In [156]:
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
#***from sklearn.model_selection import GridSearchCV
#now i will split train data in XTRAIN I XVAL

In [157]:
#kf.split( )

# k = 5
# kf = KFold(n_splits=5, random_state=None)
# for train_index , test_index in kf.split(X):
#     X_train , X_val = X.iloc[train_index,:],X.iloc[test_index,:]
#     Y_train , Y_val = Y[train_index] , Y[test_index]
 
#po indeksima dijelis xcili train i y kolonu

In [159]:
#X_train.shape

In [163]:
Y_train.shape

NameError: name 'Y_train' is not defined

In [164]:
#X_val.shape

In [165]:
#Y_val.shape

In [166]:
# kf = KFold(n_splits=5)
# for train_index, val_index in kf.split(X,Y):
#     print("TRAIN:", train_index, "VAL:", val_index)

In [168]:
#kf.split(X)
#ok je ako generator object

In [169]:
# IDEJA: ACCURACY BI SE TREBAO RACUNAT ZA SVAKI FOLD TJ SVAK ISPLIT OD 5 SPLITOVA ZATO FOR LOOP I TO ROB MULLA RADI U CROSS VAL
#kf = KFold(n_splits=5, shuffle=True, random_state=12)

# X_train, X_val, Y_train, Y_val= train_index , test_index in kf.split(X):
#                               X_train , X_val = X.iloc[train_index,:],X.iloc[test_index,:]
#                               Y_train , Y_val = Y.iloc[train_index] , Y.iloc[test_index]

# fold=0  
# accuracy_list = [] 
# for train_index , val_index in kf.split(X,Y):
#         X_train = X.loc[train_index]
#         Y_train = Y.loc[train_index]  
#         X_val = X.loc[val_index]
#         Y_val = Y.loc[val_index]
# # print(X_train)
# # print(Y_train)
# # print(X_val)
# # print(Y_val)

# # # #fitting model through loop
# logreg = LogisticRegression(random_state=0, max_iter=1000)
  
# logreg.fit(X_train, Y_train)
# # # #primjenjuje nauceno iz traina i bez da na varijabli
# yhatval = logreg.predict(X_val)
# accuracy_score(Y_val, yhatval)

# print(f" Fold {fold}")
# print(f" Accuracy score for fold {fold} is {accuracy_score}")

# fold += 1 
# accuracy_list.append(accuracy_score)

In [170]:
#  print(accuracy_list)

In [171]:
#iz https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
>>> import numpy as np
>>> from sklearn.model_selection import KFold
#>>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#>>> y = np.array([1, 2, 3, 4])
>>> kf = KFold(n_splits=5)
>>> kf.get_n_splits(X,Y)
#2
>>> print(kf)
KFold(n_splits=5, random_state=None, shuffle=False)
>>> for i, (train_index, val_index) in enumerate(kf.split(X)):
...     print(f"Fold {i}:")
...     print(f"  Train: index={train_index}")
...     print(f"  Test:  index={val_index}")


KFold(n_splits=5, random_state=None, shuffle=False)
Fold 0:
  Train: index=[179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376
 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394
 395 396 397 398 399 400 401 402 403 404 405 406 

In [182]:
i=0  
accuracy_list = [] 
for train_index , val_index in kf.split(X,Y):
        X_train = X.loc[train_index]
        Y_train = Y.loc[train_index]  
        X_val = X.loc[val_index]
        Y_val = Y.loc[val_index]
# print(X_train)
# print(Y_train)
# print(X_val)
# print(Y_val)

# # #fitting model through loop
        logreg = LogisticRegression(random_state=0, max_iter=1000)
  
        logreg.fit(X_train, Y_train)
# # #primjenjuje nauceno iz traina i bez da na varijabli
        yhatval = logreg.predict(X_val)
        a = (accuracy_score(Y_val, yhatval)) 
        print("\n")
        print(f" Fold {i}")
        print(f" Coefficients are {logreg.coef_} and intercept is {logreg.intercept_}")
        print(f" Accuracy score for fold {i} is {a}")

        i += 1 
        accuracy_list.append(a)
       



 Fold 0
 Coefficients are [[-1.15309915 -2.50021241 -2.0562877  -0.17860842 -0.10485149  0.58064524
   0.07917906 -0.31085682]] and intercept is [5.10040157]
 Accuracy score for fold 0 is 0.7988826815642458


 Fold 1
 Coefficients are [[-1.1270637  -2.64331846 -2.06542889 -0.37243808 -0.05405359  0.30594575
   0.06612782 -0.25154393]] and intercept is [5.0135347]
 Accuracy score for fold 1 is 0.7808988764044944


 Fold 2
 Coefficients are [[-0.98071015 -2.59120019 -2.22896665 -0.27067437 -0.08742246  0.85721538
   0.06801405 -0.25899601]] and intercept is [4.71823688]
 Accuracy score for fold 2 is 0.7808988764044944


 Fold 3
 Coefficients are [[-0.91369654 -2.65485485 -2.37212891 -0.35533338  0.12962457  0.31793703
   0.0459015  -0.20795876]] and intercept is [4.5092063]
 Accuracy score for fold 3 is 0.7528089887640449


 Fold 4
 Coefficients are [[-0.99408825 -2.56607359 -1.76465584 -0.2235587  -0.13275209  0.15861535
   0.03962622 -0.20860153]] and intercept is [4.56676842]
 Accur

In [183]:
print(accuracy_list)

[0.7988826815642458, 0.7808988764044944, 0.7808988764044944, 0.7528089887640449, 0.8426966292134831]


In [184]:
#fold 4 has the best score!

In [185]:
#verzija sa train test split
#aktiviraj****X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.2, random_state = 42)


In [ ]:
#aktiviraj****
#logreg = LogisticRegression(random_state=0, max_iter=1000)
# logreg.fit(X_train, Y_train)
#primjenjuje nauceno iz traina i bez da na varijabli

In [ ]:
#aktiviraj****yhatval = logreg.predict(X_val)

In [ ]:
#aktiviraj****accuracy_score(Y_val, yhatval)

In [ ]:
submission = pd.read_csv('gender_submission.csv')
submission.head(5)

In [ ]:
yhatsubmission = logreg.predict(titanictest)

#za predaju submission

In [ ]:
#accuracy_score(Y_given, yhatsubmission)

#Y_given radi accuracy pogledaj treba li numpy
#oba numpy 

print(type(yhatsubmission))
#Y_val.to_numpy()
print(type(Y_val))


In [ ]:
#za confusion matrix sam provjerila je li gore array i je u oba
def plot_confusion_matrix(Y_val, yhatval):
    
    cm = confusion_matrix(Y_val, yhatval)
    ax = plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax);
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix');
    ax.xaxis.set_ticklabels(['did not survive', 'survived'])
    ax.yaxis.set_ticklabels(['did not survive', 'survived'])

plot_confusion_matrix(Y_val, yhatval)   
#y_val was made on 0,2*titanictrain (20%*891 na traintestsplit uzela)so it was 179 units
#it is a sum of 88+17+20+54

In [ ]:
#od 74 prezivjela, predvidjeli smo 54 prezivjela, a 20 smo rekli da nece prezivit a jesu, znaci tocnost 54/74
#od 105 umrlih, za 88 smo rekli da nece prezivit i nisu, a za 17 smo rekli da ce prezivit a nisu, tocnost 88/105
#neka sredina tocnosti izmedju te dvije od 0,83 i 0,77 je 0,80 tocnost
#confusion matrix mi iz rezultata y kolone jedinice cita kao true a nule kao false tj 01 gore i 01 udesno


In [ ]:
#another way for confusion matrix

from sklearn import metrics
metrics.confusion_matrix(Y_val, yhatval)

from sklearn import metrics

#create confusion matrix
c_matrix = metrics.confusion_matrix(Y_val, yhatval)

#print confusion matrix
print(c_matrix)

In [ ]:
#my_array = np.array([[11,22,33],[44,55,66]])
#df = pd.DataFrame(my_array, columns = ['Column_A','Column_B','Column_C'])
#print(df)
#primjer: iz array u pandas



yhatsubmission = pd.DataFrame(yhatsubmission, columns = ['Yhatsubmission'])

print(yhatsubmission)
print(type(yhatsubmission))


In [ ]:
#yhatsubmission_top = yhatsubmission.head()
#yhatsubmission_top
for col in yhatsubmission.columns:
    print(col)
    #ime kolone za svaki slucaj

In [ ]:
#submission = pd.read_csv('gender_submission.csv')
#ovo je tablica za predaju, isti brojevi putnika kao i u test dijelu excela

In [ ]:
submission= submission.drop(['Survived'],axis=1)
print(submission)
#on 901 we see that in original submission table person didnt survive and down in new version with predicted values
#on 901 we predicted 901 to survive

In [ ]:
extracted_col = yhatsubmission['Yhatsubmission']
 
display(extracted_col)

In [ ]:
submission.insert(1, 'Yhatsubmission', extracted_col)
 
display(submission)

In [ ]:
#Feature importance shuffle

In [ ]:
#my_array = np.array(X_val)

In [ ]:
X_val.head()
#pandas dataframe

In [ ]:
print(type(X_val))
#Y_val je array(pandas core series provjerila je array)

In [ ]:
print(type(Y_val))

In [ ]:

#submission.to_csv('TitanicpredajafeatKate2023.csv')

In [ ]:
#***first column permutation
X_val1 = X_val.copy()
#panda for permutation

In [ ]:
X_val1.head()

In [ ]:
X_val1['Pclass'] = np.random.permutation(X_val1['Pclass'])

In [ ]:
X_val1.head()
#i see permutation in the first column

In [ ]:
X_val1.to_numpy()

In [ ]:
yX_val1 = logreg.predict(X_val1)

In [ ]:
a1 = accuracy_score(Y_val, yX_val1)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val1
importancePclass = 0.7932960893854749-a1
print (importancePclass)

In [ ]:
#***second column permutation
X_val2 = X_val.copy()
X_val2.head()
#panda for permutation
X_val2['Sex'] = np.random.permutation(X_val2['Sex'])
X_val2.head()

In [ ]:
X_val2.to_numpy()

In [ ]:
yX_val2 = logreg.predict(X_val2)

In [ ]:
a2 = accuracy_score(Y_val, yX_val2)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val2
importanceSex = 0.7932960893854749-a2
print (importanceSex)

In [ ]:
#***third column permutation
X_val3 = X_val.copy()
X_val3.head()
#panda for permutation
X_val3['Age'] = np.random.permutation(X_val3['Age'])
X_val3.head()

In [ ]:
X_val3.to_numpy()

In [ ]:
yX_val3 = logreg.predict(X_val3)

In [ ]:
a3 = accuracy_score(Y_val, yX_val3)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val3
importanceAge = 0.7932960893854749-a3
print (importanceAge)

In [ ]:
#***fourth column permutation
X_val4 = X_val.copy()
X_val4.head()
#panda for permutation
X_val4['SibSp'] = np.random.permutation(X_val4['SibSp'])
X_val4.head()

In [ ]:
X_val4.to_numpy()

In [ ]:
yX_val4 = logreg.predict(X_val4)

In [ ]:
a4 = accuracy_score(Y_val, yX_val4)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val4
importanceSibSp = 0.7932960893854749-a4
print (importanceSibSp)

In [ ]:
#***fifth column permutation
X_val5 = X_val.copy()
X_val5.head()
#panda for permutation
X_val5['Parch'] = np.random.permutation(X_val5['Parch'])
X_val5.head()

In [ ]:
X_val5.to_numpy()

In [ ]:
yX_val5 = logreg.predict(X_val5)

In [ ]:
a5 = accuracy_score(Y_val, yX_val5)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val5
importanceParch = 0.7932960893854749-a5
print (importanceParch)

In [ ]:
#***sixth column permutation
X_val6 = X_val.copy()
X_val6.head()
#panda for permutation
X_val6['Fare'] = np.random.permutation(X_val6['Fare'])
X_val6.head()

In [ ]:
X_val6.to_numpy()

In [ ]:
yX_val6 = logreg.predict(X_val6)

In [ ]:
a6 = accuracy_score(Y_val, yX_val6)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val6
importanceFare = 0.7932960893854749-a6
print (importanceFare)

In [ ]:
#***seventh column permutation
X_val7 = X_val.copy()
X_val7.head()
#panda for permutation
X_val7['Cabin'] = np.random.permutation(X_val7['Cabin'])
X_val7.head()

In [ ]:
X_val7.to_numpy()

In [ ]:
yX_val7 = logreg.predict(X_val7)

In [ ]:
a7 = accuracy_score(Y_val, yX_val7)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val7
importanceCabin = 0.7932960893854749-a7
print (importanceCabin)

In [ ]:
#***eighth column permutation
X_val8 = X_val.copy()
X_val8.head()
#panda for permutation
X_val8['Embarked'] = np.random.permutation(X_val8['Embarked'])
X_val8.head()

In [ ]:
X_val8.to_numpy()

In [ ]:
yX_val8 = logreg.predict(X_val8)

In [ ]:
a8 = accuracy_score(Y_val, yX_val8)

In [ ]:
#importancePclass= accuracy score X_Val-accuracy score X_val8
importanceEmbarked = 0.7932960893854749-a8
print (importanceEmbarked)

In [ ]:
dictionary_feature_importance = {'Pclass': [importancePclass], 'Sex':[importanceSex], 'Age': [importanceAge], 'SibSp':[importanceSibSp], 'Parch':[importanceParch], 'Fare':[importanceFare], 'Cabin': [importanceCabin], 'Embarked':[importanceEmbarked]}

In [ ]:
feature_importance = pd.DataFrame(dictionary_feature_importance)
feature_importance.head()

In [ ]:
#We can see that the strongest impact has variable Sex
#spremala sam varijable accuracy u posebne jer iz svakog novog refresha mala razlika u accuracy pa da ne bude razlika 
#u prenesenom oduzimanju niže

In [ ]:

#submission.to_csv('TitanicCROSS VAL Kate2023 pravi.csv')